In [31]:
%matplotlib inline
import tessitura_tools as tt
import importlib
from ipywidgets import widgets
from IPython.display import display, HTML, FileLink
from functools import partial
import warnings
from pandas import DataFrame, to_datetime, read_csv
from numpy import round as nround
from matplotlib.figure import Figure
from simplecrypt import decrypt # package is called simple-crypt
from io import StringIO
from urllib.request import urlretrieve
import seaborn as sb
from zipfile import ZipFile
from glob import glob
warnings.filterwarnings('ignore')

importlib.reload(tt);

In [32]:
interface_built = False

def decrypt_file(*args):
    
    # This function reads the inputted passwords and uses it to try and decrypt
    # the data file. A failed password raises an exception which is used to 
    # reset the password field with an error message.
    
    global data
    global tt_pricetype
    pw_status.value = 'Loading...'
    file = open('file.enc','rb').read() # Ticket data
    file2 = open('file2.enc', 'rb').read() # Price type data
    try:
        CSVplaintext = decrypt(pw_field.value, file).decode('utf8')
        file_like = StringIO(CSVplaintext)
        data = read_csv(file_like).drop('Unnamed: 0', axis=1) 
        
        CSVplaintext2 = decrypt(pw_field.value, file2).decode('utf8')
        file_like2 = StringIO(CSVplaintext2)
        tt_pricetype = read_csv(file_like2).drop('Unnamed: 0', axis=1)
        
        data['Perf date'] = to_datetime(data['Perf date'])
        data['Order date'] = to_datetime(data['Order date'])
        
        tt_pricetype['Perf date'] = to_datetime(tt_pricetype['Perf date'])
        tt.set_tt_pricetype(tt_pricetype)
        
        updated_str = str(max(data['Order date']))[0:10]
        updated_str2 = str(max(tt_pricetype['Perf date']))[0:10]
        
        # Unzip dashboard charts
        with ZipFile('dash.zip', 'r') as zip_ref:
            zip_ref.extractall('unzipped_dashboard/')

        file_date_status.value = 'Tickets updated through ' + updated_str + '. Price types updated through ' + updated_str2 + '.'
        pw_field.value = ''
        login_output.clear_output()
        if not interface_built:
            show_interface()
            show_dashboard()
    except:
        pw_field.value = ''
        pw_status.value = 'Password is not correct'
    

pw_field = widgets.Password(layout=widgets.Layout(width='175px'))
pw_field.on_submit(decrypt_file)
pw_submit_button = widgets.Button(description='Submit',layout=widgets.Layout(width='75px'))
pw_submit_button.on_click(decrypt_file)
pw_status = widgets.Label('', layout=widgets.Layout(width='150px'))
pw_label = widgets.Label('Password:', layout=widgets.Layout(width='75px'))
pw_login_box = widgets.HBox([pw_label,pw_field,pw_submit_button, pw_status], layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
login_output = widgets.Output()
display(login_output)
# Download the latest data file from Dropbox
with login_output:
    display(HTML('<h1>FWMSH QuickData</h1>'))
    display(HTML('Loading data...'))
url = 'https://dl.dropboxusercontent.com/s/mwqvbwadqnys8eh/museum_data.enc?dl=0' # General data
url2 = 'https://www.dropbox.com/s/jbj7c56h62a2m88/museum_data2.enc?dl=1' # Price type data
url3 = 'https://dl.dropboxusercontent.com/s/1ukokcpqddjqr0q/dashboard_charts.zip?dl=0' # Dashboard charts

urlretrieve(url, './file.enc')
urlretrieve(url2, './file2.enc')
urlretrieve(url3, './dash.zip')

# Once loaded, show the password prompt
login_output.clear_output(wait=True)
with login_output:
    display(HTML('<h1>FWMSH QuickData</h1>'))    
    #display(HTML('<H3><font color="red">This data will be out of date until Morgan returns on May 24'))
    display(pw_login_box)

Output()

In [33]:
output = widgets.Output(layout={'border': '1px solid black'})
output

Output(layout=Layout(border='1px solid black'))

In [34]:
#data = tt.load_data('daily_charts')

# The context holds a reference to the current result in the
# output window
context = None
sort_column = 'Perf date'
sort_ascending = False

def do_search(*args):
    
    global context
    global sort_column
    global sort_ascending
    output.clear_output()        
    
    # Select venue
    venue_option = ''
    if venue_field.value != 'All':
        venue_option = venue_field.value
        
    # Select audience
    audience_option = ''
    if audience_field.value != 'All':
        audience_option = audience_field.value
        
    # Day of week
    weekday = False
    weekend = False
    day_of_week = -1
    if weekday_field.value == 'Weekdays':
        weekday=True
    elif weekday_field.value == 'Weekends':
        weekend=True
    elif weekday_field.value == 'Monday':
        day_of_week = 0
    elif weekday_field.value == 'Tuesday':
        day_of_week = 1
    elif weekday_field.value == 'Wednesday':
        day_of_week = 2
    elif weekday_field.value == 'Thursday':
        day_of_week = 3
    elif weekday_field.value == 'Friday':
        day_of_week = 4
    elif weekday_field.value == 'Saturday':
        day_of_week = 5
    elif weekday_field.value == 'Sunday':
        day_of_week = 6
    
    
    # Group by
    group = ''
    if group_field.value == 'Show':
        group = 'show'
    elif group_field.value == 'Day':
        group = 'd'
    elif group_field.value == 'Week':
        group = 'w'
    elif group_field.value == 'Month':
        group = 'm'
    elif group_field.value == 'Calendar Year':
        group = 'Y'
    elif group_field.value == 'Fiscal Year':
        group = 'A-SEP'

    try:
        result = data.tt.search(name=name_field.value,
                               date=[min_date_field.value, max_date_field.value],
                               time=time_field.value,
                               tickets=tickets_low.value+':'+tickets_high.value,
                               revenue=revenue_low.value+':'+revenue_high.value,
                               audience=audience_option,
                               group=group,
                               venue=venue_option,
                               day_of_week=day_of_week,
                               weekday=weekday,
                               weekend=weekend)
        #if (sort_field.value == 'Newest first') and ('Perf date' in result):
        #    result = result.sort_values('Perf date', ascending=False)
        #elif (sort_field.value == 'Oldest first') and ('Perf date' in result):
        #    result = result.sort_values('Perf date', ascending=True)
        #elif sort_field.value == 'A-Z':
        #    result = result.sort_values('Description', ascending=True)
        #elif sort_field.value == 'Z-A':
        #    result = result.sort_values('Description', ascending=False)
        if sort_column in result:
            result = result.sort_values(sort_column, ascending=sort_ascending)
        context = result       
    except Exception as e:
        #print(e)
        result = DataFrame()
        with output:
            display(HTML('Search error: a common error is bad formatting for the time field.'))

    with output:        
        # Print the number of results
        display(HTML(str(len(result)) + ' results'))

        # Show the contexual buttons if conditions are correct.
        if len(result) > 0:
            add_context_buttons()
            add_sort_buttons()
            with output:
                display(result)
    
    return(result)
    
def reset_search(*args):
    
    # Function to clear the search fields and the output
    
    global sort_column
    global sort_ascending
    
    sort_column = 'Perf date'
    sort_ascending = False
    
    name_field.value = ''
    min_date_field.value = None
    max_date_field.value = None
    time_field.value = ''
    venue_field.value = 'All'
    audience_field.value = 'All'
    weekday_field.value = 'All'
    group_field.value = 'None'
    tickets_low.value = ''
    tickets_high.value = ''
    revenue_low.value = ''
    revenue_high.value = ''
    #sort_field.value = 'Newest first'
    
    search_tabs.selected_index = 0
    output.clear_output()
    
def add_sort_buttons(*args):
    
    # Function to figure out which columns are sortable and create buttons for them    
    # We have a DataFrame
    
    global sort_column
    
    if type(context) == DataFrame:
        button_bar = widgets.HBox(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
        button_bar.children += (widgets.Label(value='Sort by:'),)
        for key in context:
            if len(context[key].unique()) > 1:
                if key == sort_column:
                    icon = 'check'
                else:
                    icon = ''
                but = widgets.Button(description=key, icon=icon, layout=widgets.Layout(width='100px'))
                but.on_click(partial(update_sort,key))
                button_bar.children += (but,)
                
        # Display the buttons
        with output:
            display(button_bar)        

def update_sort(col, *args):
    
    # Called when a sort button is pressed to change the sort behavior.
    
    global sort_column
    global sort_ascending
    global context
    
    if sort_column == col:
        sort_ascending = not sort_ascending # Flip between True and False
    else:
        sort_column = col
        
    if sort_column in context:
        context = context.sort_values(sort_column, ascending=sort_ascending)
        output.clear_output(wait=True)
        with output:
            # Print the number of results
            display(HTML(str(len(context)) + ' results'))
            # Show context-aware buttons
            add_context_buttons()
            add_sort_buttons()
            # Show the sorted search
            display(context)


def add_context_buttons(*args):
    
    global context
    
    # Create a bar to hold the buttons
    button_bar = widgets.HBox(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
    
    # We have a DataFrame
    if type(context) == DataFrame:
        if (group_field.value == 'None') and ('Description' in context):
            button_bar.children += (sales_plot_button,)
        elif (group_field.value in ['Day','Week','Month','Calendar Year', 'Fiscal Year']) and (len(context) > 1):
            button_bar.children += (revenue_plot_button, tickets_plot_button)
        save_button.description = 'Save as Excel'
        button_bar.children += (save_button,)
        if len(context) > 1:
            button_bar.children += (sum_button, mean_button)
        if ('Perf date' in context) and (group_field.value == 'None') and (min(context['Perf date']) > to_datetime('2018-01-01')):
            button_bar.children += (age_button, member_button, group_button)
            
    # We have a plot
    if type(context) == Figure:        
        save_button.description = 'Save Graph'
        button_bar.children += (save_button,)
                
    # Display the buttons
    with output:
        display(button_bar)        
    
def make_revenue_plot(*args):
    
    global context
    
    tt.set_plot_size(16,10)
    
    if type(context) == DataFrame:
        output.clear_output()
        plot = context.tt.create_revenue_chart()
        context = plot
        add_context_buttons()
        with output:
            display(plot)
        
def make_tickets_plot(*args):
    
    global context
    
    tt.set_plot_size(16,10)
    
    if type(context) == DataFrame:
        output.clear_output()
        plot = context.tt.create_tickets_chart()
        context = plot
        add_context_buttons()
        with output:
            display(plot)

def make_sales_plot(*args):
    
    global context
    
    tt.set_plot_size(16,10)

    if type(context) == DataFrame:
        output.clear_output()
        if len(context) < 10:
            plot = data.tt.create_sales_chart(context, end_on_event=True, annotate=True)
            context = plot
            add_context_buttons()
            with output:
                display(plot)
        else:
            with output:
                print('Error: Cannot create a sales graph for more than ten results. You have ' + str(len(context))+'.')

def age_split(*args):
    
    global context
    
    # We have a DataFrame
    if type(context) == DataFrame:
        output.clear_output(wait=True)
        with output:
            split = context.tt.get_age_data()
            if 'Child tickets' in split:
                df = DataFrame()
                df['Age'] = ['Child', 'Adult']
                df['Tickets'] = [split['Child tickets'], split['Adult tickets']]
                df['Revenue'] = [split['Child revenue'], split['Adult revenue']]
                print(str(nround(100*split['Adult fraction'],1)) + '% of tickets matching this search were sold to adults.')
                display(df)
                context = df
                add_context_buttons()
            else:
                print('Age data not available for any events in this search.')
            
def group_split(*args):
    
    global context
    
    # We have a DataFrame
    if type(context) == DataFrame:
        output.clear_output(wait=True)
        with output:
            split = context.tt.get_group_data()
            if 'Group tickets' in split:
                df = DataFrame()
                df['Type'] = ['Group', 'Regular']
                df['Tickets'] = [split['Group tickets'], split['Regular tickets']]
                df['Revenue'] = [split['Group revenue'], split['Regular revenue']]
                print(str(nround(100*split['Group fraction'],1)) + '% of tickets matching this search were sold to groups.')
                display(df)
                context = df
                add_context_buttons()
            else:
                print('Group data not available for any events in this search.')

def member_split(*args):
    
    global context
    
    # We have a DataFrame
    if type(context) == DataFrame:
        output.clear_output(wait=True)

        with output:
            split = context.tt.get_member_data()
            if 'Member tickets' in split:
                df = DataFrame()
                df['Type'] = ['Member', 'Non-member']
                df['Tickets'] = [split['Member tickets'], split['Guest tickets']]
                df['Revenue'] = [split['Member revenue'], split['Guest revenue']]
                print(str(nround(100*split['Member fraction'],1)) + '% of tickets matching this search were sold to members.')
                display(df)
                context = df
                add_context_buttons()
            else:
                print('Member data not available for any events in this search.')
                
def master_save(*args):
        
    # We have a DataFrame
    if type(context) == DataFrame:
        save_xlsx(*args)
    
    # We have a plot
    if type(context) == Figure:
        save_plot(*args)
        
def save_plot(*args):
    if type(context) == Figure:
        context.savefig('saved_graph.png', dpi=300, bbox_inches='tight')
        output.clear_output()
        with output:
            display(HTML('Click the following link to download the file:'))
            display(FileLink("saved_graph.png"))
            
def save_xlsx(*args):
    #result = do_search().reset_index()
    if type(context) == DataFrame:
        context.to_excel('saved_data.xlsx')
        output.clear_output()
        with output:
            display(HTML('Click the following link to download the file:'))
            display(FileLink("saved_data.xlsx"))

def choose_autocomplete(*args, value):
    
    # This is called by button.on_click when a user selects
    # an autocomplete choice
    
    name_field.value = value
    autocomplete_preview.clear_output()
    do_search()
            
def autocomplete(*args, select=False):
    
    # Function to populate a list of matching shows as a user
    # types in the name field. Setting select=True will set
    # the name field to the first selection in the list.
    
    try:
        partial_matches = data[data['Description'].str.lower().str.contains(name_field.value.lower())]['Description'].unique().tolist()
    except:
        partial_matches = list()
    
    # Special hack for people who type exhibits when they want general admission
    if (name_field.value.lower() in "exhibits") and ('General Admission' not in partial_matches):
        partial_matches.insert(0,'General Admission')
        
    if (select == False) and (name_field.value not in partial_matches):
        
        # Create a button for the first five partial matches
        if name_field.value != '':
            autocomplete_preview.clear_output(wait=True)
            autocomplete_bar = widgets.HBox(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
            for item in partial_matches[0:min(len(partial_matches), 5)]:
                temp_button = widgets.Button(description=item, layout=widgets.Layout(width='18%'))
                temp_button.on_click(partial(choose_autocomplete, value=item))
                autocomplete_bar.children += (temp_button,)

            with autocomplete_preview:
                display(autocomplete_bar)
        else:
            autocomplete_preview.clear_output()


    elif (select == True) and (len(name_field.value) > 0) and (len(partial_matches) > 0):
        autocomplete_preview.clear_output()
        name_field.value = partial_matches[0]
        do_search()
    else:
        autocomplete_preview.clear_output()

def data_sum(*args):
    global context
    if type(context) == DataFrame:       
        context = DataFrame(context.sum()).T[['Tickets', 'Revenue']]
        output.clear_output()
        add_context_buttons()
        with output:
            display(context)
            
def data_mean(*args):
    global context
    if type(context) == DataFrame:       
        context = DataFrame(context.mean()).T
        output.clear_output()
        add_context_buttons()
        with output:
            display(context)
            
def show_dashboard():
    # Dynamically build a table to display the images
    
    files = glob('unzipped_dashboard/dashboard_charts/*.png')
    if len(files) < 2: # This makes sure a single image isn't too big and no images don't cause problems.
        files.append('spacer_image.png')
    i = 0
    command = '<table>'
    while i < len(files):
        # Add a row
        command += '<tr>'
        for j in range(2): # Cycle spots in a row
            # Add an element
            if i < len(files):
                command += '<th><img src='+files[i]+'></th>'
                i += 1
        # End the row
        command += '</tr>'
    command += '</table>'
    with dashboard_output:
        display(HTML(command))


In [35]:
file_date_status = widgets.Label('')

name_field = widgets.Text(description='Name', layout=widgets.Layout(width='275px'))
name_field.observe(autocomplete, names='value')
name_field.on_submit(partial(autocomplete, select=True))
autocomplete_preview = widgets.Output(layout=widgets.Layout(width='100%',border='1px solid black'))

min_date_field = widgets.DatePicker(description='Start date', layout=widgets.Layout(width='225px'))
max_date_field = widgets.DatePicker(description='End date', layout=widgets.Layout(width='225px'))
time_field = widgets.Text(description='Time', layout=widgets.Layout(width='175px'))
time_field.on_submit(do_search)
basic_box = widgets.HBox([name_field, min_date_field, max_date_field, time_field],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))


venue_field = widgets.Dropdown(description='Venue',
                                    layout=widgets.Layout(width='200px'),
                                    options=['All', 'Noble', 'Omni'],
                                    value='All')
venue_field.observe(do_search, names='value')
audience_field = widgets.Dropdown(description='Audience',
                                    layout=widgets.Layout(width='200px'),
                                    options=['All', 'Public', 'School'],
                                    value='All')
audience_field.observe(do_search, names='value')
weekday_field = widgets.Dropdown(description='Days of week',
                                    layout=widgets.Layout(width='200px'),
                                    options=['All', 'Weekdays', 'Weekends', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                                    value='All')
weekday_field.observe(do_search, names='value')
group_field = widgets.Dropdown(description='Group by',
                                    layout=widgets.Layout(width='200px'),
                                    options=['None', 'Show', 'Day', 'Week', 'Month', 'Calendar Year', 'Fiscal Year'],
                                    value='None')
group_field.observe(do_search, names='value')
#sort_field = widgets.Dropdown(description='Sort by',
#                                    #layout={'border': '1px solid black'},
#                                    options=['Newest first', 'Oldest first', 'A-Z', 'Z-A'],
#                                    value='Newest first')
#sort_field.observe(do_search, names='value')

group_field.observe(do_search, names='value')
options_box = widgets.HBox([venue_field, audience_field, weekday_field, group_field],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))

search_button = widgets.Button(description='Search')
search_button.on_click(do_search)
reset_button = widgets.Button(description='Clear')
reset_button.on_click(reset_search)
search_box = widgets.HBox([search_button, reset_button],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))

tickets_label = widgets.Label(value='Tickets:', layout=widgets.Layout(width='60px'))
tickets_low = widgets.Text(description='Min', value='', layout=widgets.Layout(width='150px'))
tickets_low.on_submit(do_search)
tickets_high = widgets.Text(description='Max', value='', layout=widgets.Layout(width='150px'))
tickets_high.on_submit(do_search)
ticket_range_box = widgets.HBox([tickets_label, tickets_low, tickets_high],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))

revenue_label = widgets.Label(value='Revenue:', layout=widgets.Layout(width='60px'))
revenue_low = widgets.Text(description='Min', value='', layout=widgets.Layout(width='150px'))
revenue_low.on_submit(do_search)
revenue_high = widgets.Text(description='Max', value='', layout=widgets.Layout(width='150px'))
revenue_high.on_submit(do_search)
revenue_range_box = widgets.HBox([revenue_label, revenue_low, revenue_high],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))

# Contextually-aware buttons
sales_plot_button = widgets.Button(description='Make a sales graph', layout=widgets.Layout(width='150px'))
sales_plot_button.on_click(make_sales_plot)
revenue_plot_button = widgets.Button(description='Make a revenue graph', layout=widgets.Layout(width='150px'))
revenue_plot_button.on_click(make_revenue_plot)
tickets_plot_button = widgets.Button(description='Make a tickets graph', layout=widgets.Layout(width='150px'))
tickets_plot_button.on_click(make_tickets_plot)
save_button = widgets.Button(description='Save as Excel', layout=widgets.Layout(width='150px'))
save_button.on_click(master_save)
sum_button = widgets.Button(description='Sum data', layout=widgets.Layout(width='150px'))
sum_button.on_click(data_sum)
mean_button = widgets.Button(description='Average data', layout=widgets.Layout(width='150px'))
mean_button.on_click(data_mean)
age_button = widgets.Button(description='Get age split', layout=widgets.Layout(width='150px'))
age_button.on_click(age_split)
member_button = widgets.Button(description='Get member split', layout=widgets.Layout(width='150px'))
member_button.on_click(member_split)
group_button = widgets.Button(description='Get group split', layout=widgets.Layout(width='150px'))
group_button.on_click(group_split)

adv_box = widgets.HBox([ticket_range_box, revenue_range_box],
                           layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))

top_box = widgets.VBox([basic_box, autocomplete_preview],layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))

search_tabs = widgets.Tab()
search_tabs.children = [options_box, adv_box] # basic_box is actually the name, date, time. Bad nomenclature :(
search_tabs.set_title(0, 'Basic')
search_tabs.set_title(1, 'Advanced')

search_interface_box = widgets.VBox([top_box, search_tabs, search_box])

dashboard_output = widgets.Output()

interface_tabs = widgets.Tab()
interface_tabs.children = [dashboard_output, search_interface_box]
interface_tabs.set_title(0, 'Dashboard')
interface_tabs.set_title(1, 'Search')

def show_interface(*args):
    global interface_built
    #display(top_box)
    #display(search_tabs)
    #display(search_box)
    display(file_date_status)
    display(interface_tabs)
    interface_built = True

In [36]:
# Uncomment to hide code

#HTML('''<script>
#code_show=true; 
#function code_toggle() {
# if (code_show){
# $('div.input').hide();
# } else {
# $('div.input').show();
# }
# code_show = !code_show
#} 
#$( document ).ready(code_toggle);
#</script>
#<form action="javascript:code_toggle()"><input type="submit" value="Advanced: show code"></form>''')